In [1]:
import gym
import math
import collections
import random
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen = 5000)
        self.minibatch_size = 64

    def append(self, state, action, reward, next_state, terminal):
        self.buffer.append([state, action, reward, next_state, terminal])

    def sample(self):
        mini_batch = random.sample(self.buffer, self.minibatch_size)
        # mini_batch.append(self.buffer[-1])
        s_lst, action, r_lst, s_prime_lst, done_mask_lst = map(list, zip(*mini_batch))
        return torch.FloatTensor(s_lst).to(device), torch.FloatTensor(action).to(device), torch.FloatTensor(r_lst).to(device), \
                torch.FloatTensor(s_prime_lst).to(device), torch.FloatTensor(done_mask_lst).to(device)
    
    def size(self):
        return len(self.buffer)

In [3]:
class Actor(nn.Module):
    def __init__(self):
        super(Actor, self).__init__()
        self.actionNetwork = nn.Sequential(
            nn.Linear(3, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,1)
        ).to(device)
        
    def forward(self, state):
        return torch.sigmoid(self.actionNetwork(state)) * 2
    

class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.valueNetwork = nn.Sequential(
            nn.Linear(3+1, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,1)
        ).to(device)
        
    def forward(self, state, action):
        state = state.view(-1,state.shape[-1])
        action = action.view(-1,action.shape[-1])
        return self.valueNetwork(torch.cat([state, action], dim = -1))



class DDPG():
    def __init__(self):
        super(DDPG, self).__init__()
        self.actor = Actor()
        self.actor_target = Actor()
        self.critic = Critic()
        self.critic_target = Critic()
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.actor_target.load_state_dict(self.actor.state_dict())
        
        self.actionOptimizer = optim.Adam(self.actor.parameters(), lr = 0.001)
        self.valueOptimizer = optim.Adam(self.critic.parameters(), lr = 0.001)
        self.criticLoss = F.smooth_l1_loss
        self.env = gym.make('Pendulum-v1')
        self.replay_buffer = ReplayBuffer()
        self.tau = 0.001
        self.gamma = 0.9
        self.num_replay = 15
        self.reward = 0
        self.count = 0
        self.last_state = None
        self.last_action = None
        self.flag = False
        print("action space : ",self.env.action_space.shape)
        
    def train(self, epi):
        self.last_state = self.env.reset()
        
        
        while True:
            self.env.render()
            action = self.actor(torch.FloatTensor(self.last_state).to(device))
            state, reward, done, _= self.env.step(action.detach().cpu().numpy())
            self.count += 1
            self.reward += reward
            
            self.replay_buffer.append(self.last_state, action.detach().cpu().numpy(), reward, state, done)
            if self.replay_buffer.size()>self.replay_buffer.minibatch_size:
                for _ in range(self.num_replay):
                    self.optimize_network()

            if done:
                print(f'Epi : {epi}   Avg reward : {self.reward/self.count}')
                break
            self.last_state = state
        torch.cuda.empty_cache()
        self.env.close()
        
            
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            
    def optimize_network(self):
        states, actions, rewards, next_states, terminals = self.replay_buffer.sample()
        q_next_mat = self.critic_target(next_states, self.actor_target(next_states)).view(-1)
        targetQ = rewards + q_next_mat*(1-terminals)*self.gamma
    
        self.valueOptimizer.zero_grad()
        q_mat = self.critic(states, actions).view(-1)
        valueLoss = self.criticLoss(q_mat,targetQ)
        valueLoss.backward()
        self.valueOptimizer.step()
        
        self.actionOptimizer.zero_grad()
        q_mat = self.critic(states, self.actor(states)).view(-1)
        actionLoss = (-q_mat.mean()).backward()
        self.actionOptimizer.step()
        
        self.soft_update(self.critic, self.critic_target, self.tau)
        self.soft_update(self.actor, self.actor_target, self.tau)


In [4]:
model = DDPG()

for epi in range(100):
    model.train(epi)


: 

: 